In [8]:
from scipy import interpolate
from scipy.interpolate import interp1d

%run ./libs.ipynb
%run ./initialize.ipynb
%run ./NYQInterp_function.ipynb
%run ./CohSumTD_function.ipynb
%run ./signals_sec322.ipynb

AutoPhase = False
noise = 0.15
order = 5

global Ss_max
global Sfilt_max
Ss_max = 3.01e-7
Sfilt_max = 5e-3

In [31]:
def getPlot(Ss, dsampled, Sfilt, PhySig, Noversampl, BASE, SAMP, SampSig, LOG):
    rf_vec1 = dsampled/(LB/2) #np.linspace(-1,1,len(dsampled))
    rf_vec2 = np.linspace(-1000,1000,len(Sfilt))
    
    fig = plt.figure(figsize = (14,4))
    ax = fig.add_subplot(111)
    
    if SAMP:
        SAMPrange = rf_vec1 + Noversampl
        LB_rect = np.heaviside(SAMPrange,0.5)
        LB_rect[rf_vec1 > Noversampl] = 0
        trans = mtransforms.blended_transform_factory(ax.transData, ax.transAxes)
        ax.fill_between(rf_vec1, 0, 1, where=LB_rect > 0.5,
                        facecolor='red', alpha=0.15, transform=trans)
        
    if BASE:
        LBrange = rf_vec1 + 1
        LB_rect = np.heaviside(LBrange,0.5)
        LB_rect[rf_vec1 > 1] = 0
        trans = mtransforms.blended_transform_factory(ax.transData, ax.transAxes)
        ax.fill_between(rf_vec1, 0, 1, where=LB_rect > 0.5,
                        facecolor='blue', alpha=0.15, transform=trans)
        
    if SampSig:
        line1 = ax.plot(rf_vec1, Ss,'r', linewidth='2')
    
        if PhySig:
            
            line2 = ax.plot(rf_vec2[int(len(Sfilt)/2-len(Sfilt)/nptslambda/2*5):int(len(Sfilt)/2-len(Sfilt)/nptslambda/2*Noversampl)],
                        Sfilt[int(len(Sfilt)/2-len(Sfilt)/nptslambda/2*5):int(len(Sfilt)/2-len(Sfilt)/nptslambda/2*Noversampl)],'k', linewidth='1')

            line3 = ax.plot(rf_vec2[int(len(Sfilt)/2+len(Sfilt)/nptslambda/2*Noversampl):int(len(Sfilt)/2+len(Sfilt)/nptslambda/2*5)],
                        Sfilt[int(len(Sfilt)/2+len(Sfilt)/nptslambda/2*Noversampl):int(len(Sfilt)/2+len(Sfilt)/nptslambda/2*5)],'k', linewidth='1')

    else:
        if PhySig:
            line2 = ax.plot(rf_vec2, Sfilt,'k', linewidth='1')
        
    plt.ylabel('Signal, a.u.', fontsize = 15)
    plt.xlabel('$f/f_o$', fontsize = 15)
    if LOG:
        plt.ylim([-58, 2])
    else:
        plt.ylim([0, 1.35])
    plt.xlim([-5, 5])
    plt.xticks(np.arange(-5, 5, step=1))
    plt.grid(color='k', linestyle='-', linewidth=.1)
    plt.show()         
    
#def getNorm(_):
#    global Ss_max
#    global Sfilt_max
#    Ss_max = np.max(Ss)
#    Sfilt_max = np.max(Sfilt)
#    #GetNorm_button.value = False
        
def getSig(Nsampl, f_slide, Filt_slider, PhySig_box, BASE_box, SAMP_box, SampSig_box, 
           LOG_button, CohSum_button, NYQInterp_button, Norm_button, ClockPhase_slider):

    #NYQInterp = NYQInterp_button
    #CohSum = CohSum_button
    #LOG = LOG_button
    #SampSig = SampSig_box
    #SAMP = SAMP_box
    #BASE = BASE_box
    #PhySig = PhySig_box
    
    if CohSum_button == True:
        PhySig_box = False
    if NYQInterp_button == True:
        PhySig_box = False

    ph0 = ClockPhase_slider/2/np.pi
    
    FILTFACT = Filt_slider
    f = f_slide
    #FiltType = LPFtype
    #Order = order
    #FILT = LPF
    Noversampl = Nsampl
    depth0 = f*LB/2*1e6
    
    Ss, dsampled, Sfilt = signals_sec322(Noversampl, depth0, oof, ofcont, FILT, 
                                         Order, FiltType, FILTFACT, AutoPhase, ph0, IQ, noise, PhySig_box, CohSum_button, NYQInterp_button)
    
    if Norm_button:
        global Ss_max
        global Sfilt_max
        Ss_max = np.max(Ss)
        Sfilt_max = np.max(Sfilt)
    
    Ss = Ss/Ss_max #3.01e-7
    Sfilt = Sfilt/Sfilt_max #5e-3
    
    if LOG_button:
        Ss = 10*np.log10(Ss)
        Sfilt = 10*np.log10(Sfilt)
        
    getPlot(Ss, dsampled, Sfilt, PhySig_box, Noversampl, BASE_box, SAMP_box, SampSig_box, LOG_button)

In [ ]:
# depth
f_slide = widgets.FloatSlider(value = .5,
                              description = '$f/f_o$',
                              min = -2,
                              max = 2,
                              step = .1,
                              continuous_update = True,
                              layout=Layout(width='310px', margin='1px 1px'))

Filt_slider = widgets.FloatSlider(value = 3.7,
                                  min=0.7, max=5, step=0.1,
                                  description = 'LPF cutoff',
                                  readout_format='.1f',
                                  layout=Layout(width='310px', margin='0px 0px'))
# N oversampling
Nsampl = widgets.FloatText(value = 3.7,
                           description = 'Sampling',
                           min = 2,
                           max = 5,                         
                           layout=Layout(width='150px'))

# N oversampling
ClockPhase_slider = widgets.FloatSlider(value = 3.14,
                           description = 'ClockPhase',
                           min = 0,
                           max = 6.28,                         
                           layout=Layout(width='310px'))

## low-pass filter
#LPF = widgets.ToggleButton(value = True,
#                           description = 'LPF',
#                           layout=Layout(width='132px', margin='1px 5px')) 

#order = widgets.Dropdown(options = [('2',1),('3',2),('4',3),('5',4)],
#                         value = 2,
#                         description = 'Order:',
#                         layout=Layout(width='175px', margin='1px 5px'))

#LPFtype = widgets.Dropdown(options = [('Butter', 1), ('Cheby', 2)],
#                           value = 1,
#                           description = 'Type:',
#                           layout=Layout(width='175px', margin='1px 5px'))

# log
LOG_button = widgets.ToggleButton(value = False, description = 'Log', layout=Layout(width='110px', margin='13px 0px')) 

Norm_button = widgets.ToggleButton(value = False, description = 'Norm', layout=Layout(width='110px', margin='0px 0px'))
                                       

#GetNorm_button = widgets.Button(description='GetNorm', button_style = 'danger', layout=Layout(width='80px', margin='1px 0px'))



CohSum_button = widgets.ToggleButton(value = False, description = 'Coh. Sum.', 
                                     button_style='success', # 'success', 'info', 'warning', 'danger' or ''
                                     layout=Layout(width='110px', margin='0px 1px')) 

NYQInterp_button = widgets.ToggleButton(value = False, description = 'Nyq. Interp.', 
                                        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
                                        layout=Layout(width='110px', margin='13px 1px'))

PhySig_box = widgets.Checkbox(value=False, description='Physical signal', disabled=False, indent=False)

SampSig_box = widgets.Checkbox(value=True, description='Sampled signal', disabled=False, indent=False)

SAMP_box = widgets.Checkbox(value=False, description='Sampling window', disabled=False, indent=False)

BASE_box = widgets.Checkbox(value=True, description='Baseband window', disabled=False, indent=False)


In [ ]:
display(Image(filename='setup_sec322.png', width="860"))

ui1 = widgets.VBox([f_slide])
ui2 = widgets.VBox([Filt_slider])
ui3 = widgets.VBox([ClockPhase_slider, Nsampl], layout=widgets.Layout(display='flex', flex_flow='column', align_items='flex-start'))
ui4 = widgets.HBox([ui1, ui2, ui3])

ui5 = widgets.VBox([NYQInterp_button, CohSum_button], layout=widgets.Layout(display='flex', flex_flow='column', align_items='flex-end'))
ui6 = widgets.VBox([LOG_button, Norm_button], layout=widgets.Layout(display='flex', flex_flow='column', align_items='flex-start'))
ui7 = widgets.VBox([ui5, ui6], layout=widgets.Layout(display='flex', flex_flow='column', align_items='center'))

#ui3 = widgets.VBox([LPF, order, LPFtype], layout=widgets.Layout(display='flex', flex_flow='column', align_items='flex-end'))


#ui4 = widgets.VBox([NYQInterp_button, CohSum_button], layout=widgets.Layout(display='flex', flex_flow='column', align_items='center'))
                  
out = widgets.interactive_output(getSig, {'Nsampl': Nsampl, 
                                          'f_slide': f_slide, 'Filt_slider': Filt_slider,
                                          'PhySig_box': PhySig_box, 'SampSig_box': SampSig_box, 'BASE_box': BASE_box, 
                                          'SAMP_box': SAMP_box, 'LOG_button': LOG_button, 'CohSum_button': CohSum_button, 
                                          'NYQInterp_button': NYQInterp_button, 'Norm_button': Norm_button, 'ClockPhase_slider': ClockPhase_slider})

ui7 = widgets.HBox([out, ui7])


ui8 = widgets.HBox([PhySig_box, SampSig_box, SAMP_box, BASE_box])

display(ui4, ui7, ui8)


